In [52]:
import tensorflow as tf
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [53]:
import os
from dotenv import load_dotenv

load_dotenv(".env")
data=os.getenv("dataset")

df=pd.read_csv(data)

df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [54]:
#Assign Labels to classes
labels=list(df['label'].unique())

mapping=dict()

for i,j in enumerate(labels):
    mapping[j]=i

df['label']=df['label'].map(mapping)

In [55]:
#Normalize
columns=['N','P','K','temperature','humidity','ph','rainfall']

for i in columns:
    df[i]=(df[i]-np.mean(df[i]))/np.std(df[i])

In [56]:
df

,N,P,K,temperature,humidity,ph,rainfall,label
0,1.068797,-0.344551,-0.101688,-0.935587,0.472666,0.043302,1.810361,0
1,0.933329,0.140616,-0.141185,-0.759646,0.397051,0.734873,2.242058,0
2,0.255986,0.049647,-0.081939,-0.515898,0.486954,1.771510,2.921066,0
3,0.635298,-0.556811,-0.160933,0.172807,0.389805,0.660308,2.537048,0
4,0.743673,-0.344551,-0.121436,-1.083647,0.454792,1.497868,2.898373,0
...,...,...,...,...,...,...,...,...
2195,1.529390,-0.587134,-0.318922,0.228814,-0.227709,0.401395,1.352437,21
2196,1.312641,-1.163269,-0.417666,0.355720,-0.666947,-0.494413,0.445183,21
2197,1.827421,-0.617457,-0.358420,-0.293218,-0.191235,-0.138120,1.271418,21
2198,1.800327,-0.647780,-0.279425,0.129612,-0.869518,0.373904,0.431545,21


In [57]:
X,y=df.drop(['label'],axis=1),df['label']

X=np.array(X)
y=np.array(y)

In [58]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,y,train_size=0.75)


In [59]:
model=tf.keras.Sequential()

In [60]:
#Build model using sequential api
def model_build(layers=5):
   hidden_units=[35,55,45,35,22]
   
   final=tf.keras.initializers.GlorotNormal(seed=1) 
   initials=tf.keras.initializers.HeNormal(seed=1) #Initialize weights to get values with 0 means
   for i in range(layers):
       if i==0:
           model.add(tf.keras.layers.Dense(hidden_units[i],activation="relu",kernel_initializer=initials,use_bias=True,bias_initializer=initials,input_shape=(7,)))
           model.add(tf.keras.layers.Dropout(rate=0.5))
       elif i>0 and i<layers-1:
           model.add(tf.keras.layers.Dense(hidden_units[i],activation="relu",kernel_initializer=initials,use_bias=True,bias_initializer=initials))
           model.add(tf.keras.layers.Dropout(rate=0.5))
       elif i==layers-1:
           model.add(tf.keras.layers.Dense(hidden_units[i],activation="softmax",kernel_initializer=final,use_bias=True,bias_initializer=final))

In [61]:
def model_run(x_train,y_train,x_test,y_test,learning_rate=0.001,num_epochs=2000,batchSize=32,show_costs=1): #Define hyperparameters for training
    algo=tf.keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=algo,loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    learning=model.fit(x_train,y_train,epochs=num_epochs,batch_size=batchSize,verbose=show_costs,validation_data=(x_test,y_test))
    
    return learning

In [62]:
model_build()

/Users/smudge/Desktop/Code/Uni/iot-project/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 35)             │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 35)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 55)             │         1,980 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 55)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 45)             │         2,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 45)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 35)             │         1,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 35)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 22)             │           792 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,182 (28.05 KB)

 Trainable params: 7,182 (28.05 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
learning=model_run(x_train,y_train,x_test,y_test)

Epoch 1/2000
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.2492 - sparse_categorical_accuracy: 0.0293 - val_loss: 3.1230 - val_sparse_categorical_accuracy: 0.0364
Epoch 2/2000
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - loss: 3.9485 - sparse_categorical_accuracy: 0.0590 - val_loss: 3.0666 - val_sparse_categorical_accuracy: 0.0945
Epoch 3/2000
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: 3.6406 - sparse_categorical_accuracy: 0.0493 - val_loss: 3.0561 - val_sparse_categorical_accuracy: 0.1109
Epoch 4/2000
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - loss: 3.3472 - sparse_categorical_accuracy: 0.0392 - val_loss: 3.0561 - val_sparse_categorical_accuracy: 0.1109
Epoch 5/2000
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - loss: 3.2632 - sparse_categorical_accuracy: 0.0600 - val_loss: 3.0568 - val_sparse_categorical_accuracy: 0.1200
Epoch 6/2000
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 3.1407 - sparse_categorical_accuracy: 0.0564 - val_loss: 3.0517 - val_sparse_categorical_accuracy: 0

In [66]:
model.save("./model.keras")